# Creazione matrici per ogni sottopercorso in base alla mossa

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
path = "DB-Output_original.csv"

data = pd.read_csv(path)

#data = data[:9000] # posso ridurre da 35mila a 9mila senza perdere i luoghi, velocizzando l'esecuzione
data = data.loc[:, ['Initial Solution', 'Moves']]

data.head(5)

,Initial Solution,Moves
0,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'DemandBasedDestroyCustomer',..."
1,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'ShawDestroyCustomer', 'Greed..."
2,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'WorstTimeDestroyCustomer', '..."
3,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['null', 'null', 'GreedyRouteRemoval', 'Greedy..."
4,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","['RandomDestroyStation', 'ProbabilisticBestRep..."


# Lista delle mosse disponibili e applicate

A partire dal dataframe iniziale, si crea una lista chiamata "mv", dove al suo interno sono presenti tutte le diverse mosse applicate e riportate nella tabella.

In [2]:
mv = data["Moves"]
moves = []
for row in mv:
    for elem in row.split("', '"):
        moves.append(elem)

for i in range(len(moves)):
    moves[i] = moves[i].replace("['","")
    moves[i] = moves[i].replace("']","")

to_remove = "null"
moves = list(filter(lambda x: x != to_remove, moves))

moves=list(set(moves))

print("Lista delle", len(moves), "possibili e diverse mosse: \n", moves)

Lista delle 16 possibili e diverse mosse: 
 ['TimeBasedDestroyCustomer', 'DemandBasedDestroyCustomer', 'ProximityBasedDestroyCustomer', 'DeterministicBestRepairStation', 'ZoneDestroyCustomer', 'GreedyRepairCustomer', 'ProbabilisticBestRepairStation', 'WorstDistanceDestroyCustomer', 'RandomDestroyStation', 'ProbabilisticWorstRemovalCustomer', 'GreedyRouteRemoval', 'LongestWaitingTimeDestroyStation', 'ShawDestroyCustomer', 'RandomRouteDestroyCustomer', 'GreedyDestroyCustomer', 'WorstTimeDestroyCustomer']


In [3]:
data["Moves"] = data["Moves"].str.replace("[","")
data["Moves"] = data["Moves"].str.replace("]","")
data["Moves"] = data["Moves"].str.replace("'","")
data.head(5)

/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_75096/2851963077.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Moves"] = data["Moves"].str.replace("[","")
/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_75096/2851963077.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Moves"] = data["Moves"].str.replace("]","")


,Initial Solution,Moves
0,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, DemandBasedDestroyCustomer, Greedy..."
1,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, ShawDestroyCustomer, GreedyRepairC..."
2,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, WorstTimeDestroyCustomer, GreedyRe..."
3,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
4,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","RandomDestroyStation, ProbabilisticBestRepairS..."


In [4]:
user_input = input("Inserisci una stringa: ")
data = data.loc[data['Moves'].str.contains(user_input)]
data

,Initial Solution,Moves
3,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
17,"[['D0', 'C61', 'D0'], ['D0', 'C24', 'C50', 'D0...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
19,"[['D0', 'C61', 'D0'], ['D0', 'C24', 'C50', 'D0...","RandomDestroyStation, DeterministicBestRepairS..."
26,"[['D0', 'C24', 'D0'], ['D0', 'C26', 'C25', 'D0...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
32,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C91', 'C6...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
...,...,...
35369,"[['D0', 'C96', 'C67', 'C24', 'D0'], ['D0', 'C5...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
35380,"[['D0', 'C1', 'C2', 'C5', 'C21', 'C24', 'C29',...","RandomDestroyStation, DeterministicBestRepairS..."
35381,"[['D0', 'C1', 'C2', 'C5', 'C14', 'C25', 'C21',...","null, null, GreedyRouteRemoval, GreedyRepairCu..."
35385,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',...","RandomDestroyStation, DeterministicBestRepairS..."


Questi passaggi servono ad isolare i luoghi contenuti nella colonna "Initial Solution", in modo da poter costruire una matrice a seconda della mossa pressa in considerazione 

In [5]:
init_sol = data['Initial Solution']

sol = []
for row in init_sol:
    for elem in row.split("'], "):
        sol.append(elem)
        
for i in range(len(sol)):
    sol[i] = sol[i].replace("[['","")
    sol[i] = sol[i].replace("']]","")
    sol[i] = sol[i].replace("'","")
    sol[i] = sol[i].replace(",","")
    sol[i] = sol[i].replace("[","")
    
data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
data["Initial Solution"] = data["Initial Solution"].str.replace("]","")
data["Initial Solution"] = data["Initial Solution"].str.replace(",","")
data["Initial Solution"] = data["Initial Solution"].str.replace("'","")

init_sol = data["Initial Solution"]

places = []
for row in init_sol:
    for elem in row.split():
        places.append(elem)

places = np.array(places)
distinct_places = np.unique(places)


/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_75096/162803408.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_75096/162803408.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("]","")


In [7]:
# creazione dizionario  
dict = {}

# creazione dataframe per ogni indice del dizionario, dove l'indice è la lunghezza di "sol"
for index, elem in enumerate(sol):
        zeros = np.zeros([len(distinct_places), len(distinct_places)])
        dict[index] = pd.DataFrame(zeros, index=distinct_places, columns=distinct_places)

In [8]:
for index, stringa in enumerate(sol):
    
    temp = stringa
    
    split = temp.split(" ")                         
    split = np.array(split)
    
    for i in range(len(split)-1):
            dict[index].loc[split[i], split[i+1]] = 1

In [16]:
len(dict)

81649

In [13]:
# inizializzo il risultato come il primo dataframe del dizionario
df_sum = dict[0]

# itero su tutti gli altri dataframe del dizionario
for i in range(1, len(dict)):
    # sommo il dataframe corrente al risultato
    df_sum = df_sum.add(dict[i])


In [15]:
df_sum.to_csv(user_input)

In [ ]:
for i in range(len(dict)):
    dict[i] = dict[i].to_numpy()

In [15]:
# Inizializzo una variabile per memorizzare il risultato
result = None

# Creo un ciclo che iteri sulla lista di matrici
for matrix in dict.values():
    # Se è il primo ciclo, il risultato è uguale alla prima matrice
    if result is None:
        result = matrix
    # Altrimenti, sommo il risultato con la matrice corrente
    else:
        result = np.add(result, matrix)

# Stampo il risultato
print(result)


[[  0.   2. 104. ...   3.  15.   0.]
 [  7.   0.   9. ...  10.  35.   3.]
 [ 29.   9.   0. ...  19.   7.   0.]
 ...
 [  0.   0.   0. ...   1.   9.   0.]
 [  3.  78.   0. ...   0. 348.   1.]
 [  0.   0.   0. ...   0.   0.   0.]]
